In [ ]:
#| default_exp scrape

In [ ]:
#| export
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import re
import urllib
from tqdm.auto import tqdm
import pandas as pd
import fastcore.all as fc
from fastcore import docments
from datasets import load_dataset
import math
import shutil
from unstably_diffused.core import *

# Scraping

This notebook contains a series of tools for scraping Unsplash with Selenium. Scraping is particularly unfun and this makes it a little less soul destroying. It includes functions that support the full pipeline from scraping images and descriptive text to creating and uploading a hugging face dataset.

## Unsplash Scraper

Unsplash has a lot of images and helpfully has good alt text for most images, which can be used as a description. We can use Selenium to identify these elements and extract them. Using Selenium means we can move much faster than we can using the Unsplash API — the free/unregistered version only allows 50 requests an hour. If you are building large datasets, this means it will take a lifetime.

In [ ]:
#| export
class UnsplashScraper():
    def __init__(
        self, 
        search_term: str, # term to search on Unsplash
        save_path: str, # Root folder to save images and descriptions
        alt: bool=True, # If true, collects the alt text and saves to a txt file
        crop=True, # If crop, uses crop_512 function
        desc_front_decorator: str=None, # Optional text to add in front of scraped alt text
        desc_back_decorator: str=None, # Optional text to add to end of scraped alt text
    ):
        fc.store_attr()
        self.slug = "-".join(search_term.split())
        self.url = f"https://unsplash.com/s/photos/{self.slug}"
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.driver.get(self.url)
        self.it = 0
    
    def scrape(self):
        elements = self.get_elements()
        self.descs = [el.get_attribute('alt') for el in elements]
        self.img_links = [el.get_attribute('src') for el in elements]
        self.get_images()
        self.get_descriptions()
        
    def get_images(self):
        os.makedirs(f'{self.save_path}/img', exist_ok=True)
        for i, l in tqdm(enumerate(self.img_links), total=len(self.img_links)):
            try:
                img = download_img_from_url(l)
                if self.crop: 
                    crop = crop_square(img).resize((512, 512))
                    crop.save(f'{self.save_path}/img/{i}_{self.slug}.png')
                else:
                    img.save(f'{self.save_path}/img/{i}_{self.slug}.png')
            except: self.it = i
                
    def get_descriptions(self):
        fnames = [i for i in os.listdir(self.save_path)]
        self.desc_fname, file_count_list = 0, []
        for i in fnames: 
            if "descriptions" in i:
                try:
                    h = int(re.search('([\d]+)', i).groups()[0])
                    file_count_list.append(h)
                except: pass
        if len(file_count_list) != 0: self.desc_fname = max(file_count_list) + 1
        with open(f'{self.save_path}/descriptions_{self.desc_fname}.txt', 'w') as file:
            if self.desc_front_decorator: d = self.desc_front_decorator + d
            if self.desc_back_decorator: d = d + self.desc_back_decorator
            for d in self.descs: file.write(d + '\n')
            
    def make_dataframe(self):
        if '.ipynb_checkpoints' in [i for i in os.listdir(f'{self.save_path}/img')]:
            shutil.rmtree(f'{self.save_path}/img/.ipynb_checkpoints')
        im_fnames = [i for i in os.listdir(f'{self.save_path}/img')]
        file = open(f'{self.save_path}/descriptions_{self.desc_fname}.txt')
        descs = file.readlines()
        ns = [int(re.search(r"([\d]+)", nm).groups()[0]) for nm in im_fnames]
        df = pd.DataFrame({
            "file_name": im_fnames,
            'ns': ns
        })
        df = df.sort_values('ns', ascending=True)
        df.drop(columns='ns', inplace=True)
        df['description'] = descs
        return df
        
    def get_elements(self): return self.driver.find_elements(By.XPATH, '//div[contains(@class, "rJ2xz bYpwS U8eXG M5vdR")]/div/div/div/figure/div/div/div/div/a/div/div[contains(@class, "MorZF")]/img')
    def zip_elements(self): return zip(self.img_links, self.descs)

In [ ]:
#| export
def make_dataset(df, img_path, push_hub_path=None):
    df.to_csv(f'{path}/img/metadata.csv', index=False)
    dataset = load_dataset(img_path, split="train")
    if push_hub_path is not None: dataset.push_to_hub(push_hub_path)
    return dataset

In [ ]:
#| export
def download_img_from_url(url):
    request = urllib.request.Request(url)
    return Image.open( urllib.request.urlopen(request) )

## Cropping and resizing

Most Unsplash images are in odd shapes. To work well with Stable Diffusion, it is useful to crop images down to 512x512.

In [ ]:
#| export
def crop_square(img):
    """
        Takes an image, either portrait or landscape, and crops the middle 512x512
        portion. Does NOT do anything fancy like face recognition.
    """
    width, height = img.size   # Get dimensions
    delta = width-height
    delta_ = math.sqrt(delta**2)
    
    top=0 
    bottom=height 
    left=0
    right = width
    
    if delta < 0:
        top = delta_ // 2
        bottom = (height - (delta_ // 2))
    elif delta > 0:
        left = delta_ // 2
        right = (width - (delta_ // 2))
    else: pass
    
    return img.crop((left, top, right, bottom))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()